# QUIZ - 2024-08

In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db.models import *
from django.db.models.functions import *
from django.db.models.lookups import *
from service.models import *

## PART 1 - CRUD Operations
1.1 สร้างบริการ (`Service`) รายละเอียดตามด้านล่าง และเพิ่มเข้าไปในหมวดหมู่ (`ServiceCategory`) ชื่อ "Hair Care"

- Name: "Hair Shower"
- Desciption: "Cold hair shower"
- Price: 100 (Provider 1) และ 200 (Provider 2)
- Service Provider: "Provider 1" และ "Provider 2"

*Hint: ดังนั้นต้องสร้าง 2 แถวในตาราง `Service`*


(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [6]:
# CODE HERE
hc = ServiceCategory.objects.get(name="Hair Care")
provi1 = ServiceProvider.objects.get(name="Provider 1")
provi2 = ServiceProvider.objects.get(name="Provider 2")

s1 = Service.objects.create(name="Hair Shower", description="Cold hair shower", price=100, service_provider=provi1)
s2 = Service.objects.create(name="Hair Shower", description="Cold hair shower", price=200, service_provider=provi2)
hc.services.add(s1,s2)

In [7]:
# Check result
cat = ServiceCategory.objects.first()
cat.services.count()

6

1.2 ทำการแก้ไขราคาของ "Hair Shower" ของ "Provider 1" จาก 100 เป็น 300

(0.25 คะแนน)

In [9]:
# CODE HERE
hs = Service.objects.get(name="Hair Shower", service_provider__name="Provider 1")
hs.price = 300
hs.save()

In [10]:
# Check result
services = Service.objects.filter(name="Hair Shower")
for service in services:
    print(f"Service Name: {service.name}, Provider Name: {service.service_provider.name}, Price: {service.price}")

Service Name: Hair Shower, Provider Name: Provider 2, Price: 200
Service Name: Hair Shower, Provider Name: Provider 1, Price: 300


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-15 โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Massage" ของ "Provider 3" เป็น "Hair Shower" ของ "Provider 1" แทน และทำการเลื่อนวันที่นัดหมายจาก 2024-08-15 เป็น 2024-08-18

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [14]:
# CODE HERE
import datetime as dt
appoint = Appointment.objects.get(appointment_date=dt.date(2024, 8, 15), service__name="Massage", service__service_provider__name="Provider 3")
serv = Service.objects.get(name="Hair Shower", service_provider__name="Provider 1")
appoint.service = serv
appoint.appointment_date = dt.date(2024, 8, 18)
appoint.save()

In [15]:
# Check result
res = Appointment.objects.order_by("-appointment_date").first()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-18, Appoint Time: 11:30:00
Service Name: Hair Shower, Provider Name: Provider 1, Customer Name: Customer 1


1.4 ทำตามขั้นตอนดังนี้

1. สร้างผู้ให้บริการ (`ServiceProvider`) ชื่อ "Super Provider" ขึ้นมา (email: super@gmail.com, phone: 089-999-9999) และทำการเพิ่มบริการ (`Service`) ให้กับผู้ให้บริการนี้ (เทพมากสามารถทำได้ทุกบริการ) โดยทุกบริการราคาเท่ากับ 500 บาท

(1 คะแนน)

In [21]:
# CODE HERE
super_pro = ServiceProvider.objects.create(name="Super Provider", email="super@gmail.com", phone="089-999-9999")
for i in Service.objects.distinct("name").values_list("name", flat=True):
    Service.objects.create(name=i, price=500, service_provider=super_pro)

In [22]:
# Check result
for s in Service.objects.filter(service_provider__name="Super Provider"):
    print(f"Service Name: {s.name}, Price: {s.price}, Provider: {s.service_provider.name}")

Service Name: Hair Shower, Price: 500, Provider: Super Provider
Service Name: Haircut, Price: 500, Provider: Super Provider
Service Name: Manicure, Price: 500, Provider: Super Provider
Service Name: Massage, Price: 500, Provider: Super Provider


## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่นัดหมาย (`Appointment.appointment_date`) ในช่วงวันที่ 2024-08-05 ถึง 2024-08-10 ของ "Service Provider 1"

**Expected Output**

จะต้องมี 5 รายการ

(0.5 คะแนน)

In [7]:
# CODE HERE
import datetime as dt
Appointment.objects.filter(appointment_date__range=(dt.date(2024, 8, 5), dt.date(2024, 8, 10)), service__service_provider__name="Provider 1").count()

5

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
```

In [25]:
# CODE HERE
import datetime as dt
for i in Appointment.objects.filter(created_at__year=2024, created_at__month=8, created_at__day__range=(1, 2)).values("service__name", "service__service_provider__name").distinct("service__name", "service__service_provider__name"):
    print("Name: %s, Provider: %s"%(i['service__name'], i['service__service_provider__name']))

Name: Haircut, Provider: Provider 1
Name: Haircut, Provider: Provider 2
Name: Haircut, Provider: Provider 4
Name: Manicure, Provider: Provider 1
Name: Manicure, Provider: Provider 2
Name: Manicure, Provider: Provider 3
Name: Massage, Provider: Provider 1
Name: Massage, Provider: Provider 2


2.3 ให้แสดงผลข้อมูลลูกค้า (`Customer`) ดังในตัวอย่าง

```python
[
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [34]:
# CODE HERE
import json
print(json.dumps(list(Customer.objects.values("name", "phone").annotate(appointment_num=Count("appointment")).order_by("-name")), indent=4, sort_keys=False))

[
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]


2.4 ให้แสดงข้อมูลผู้ให้บริการ (`ServiceProvider`) ดังนี้

*หมายเหตุ: Appointment Num หมายถึงจำนวนนัดหมายที่ผู้ให้บริการนั้นมีการได้รับนัด และ Appointed Service หมายถึงบริการที่มีการนัดหมายของผู้ให้บริการนั้นๆ*

(1 คะแนน)

**Expected Output**

```python
Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut
```

In [52]:
# CODE HERE
for i in ServiceProvider.objects.values("name").annotate(appointment_num=Count("service__appointment")).order_by("name"):
    print("Name: %s, Appointment Num: %d, Appointed Services: "%(i['name'], i['appointment_num']), end="")
    print(*Appointment.objects.filter(service__service_provider__name=i['name']).values_list("service__name", flat=True), sep=", ")

Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Manicure, Manicure, Massage, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Manicure, Manicure, Massage
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut


2.5 ให้หาว่าบริการ (`Service`) ไหนของผู้บริการไหนได้รับความนิยมสูงสุด (มีจำนวนนัดหมายมากที่สุด) และ นิยมน้อยที่สุด (มีจำนวนนัดหมายน้อยที่สุด)

(0.5 คะแนน)

**Expected Output**

```python
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Massage, Provider: Provider 4, Appointment Num: 0
Name: Manicure, Provider: Provider 4, Appointment Num: 0
Name: Haircut, Provider: Provider 3, Appointment Num: 0
```

In [54]:
# CODE HERE
for i in Service.objects.values("name", "service_provider__name").annotate(appoint_num=Count("appointment")).order_by("-appoint_num"):
    print("Name: %s, Provider: %s, Appointment Num: %d"%(i['name'], i['service_provider__name'], i['appoint_num']))

Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1
Name: Manicure, Provider: Provider 4, Appointment Num: 0
Name: Haircut, Provider: Provider 3, Appointment Num: 0
Name: Massage, Provider: Provider 4, Appointment Num: 0


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อบริการ (`Service.name`) เป็นตัวใหญ่ (uppercase) และราคา (`Service.price`) เป็น 2 เท่า และเรียงลำดับตามราคา

(0.5 คะแนน)

**Expected Output**

```python
Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
```

In [56]:
# CODE HERE
for i in Service.objects.annotate(new_name=Upper("name")).annotate(new_price=F("price")*2):
    print(i)

Haircut by Provider 1
Manicure by Provider 1
Massage by Provider 1
Haircut by Provider 2
Manicure by Provider 2
Massage by Provider 2
Haircut by Provider 3
Manicure by Provider 3
Massage by Provider 3
Haircut by Provider 4
Manicure by Provider 4
Massage by Provider 4
